In [7]:
import numpy as np

from constants import FLOAT_DTYPE, INT_DTYPE, EPOCHS
from Network import CNN
from Layers import BatchNorm, Conv2D, Dropout, Relu, Softmax
from dataset import get_2D_normalised

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
(train_data, train_labels), (test_data, test_labels) = get_2D_normalised()

In [10]:
# Convert labels to one-hot encoded vectors
num_classes = 10
train_labels_enc = np.eye(num_classes)[train_labels].reshape(-1, num_classes)
test_labels_enc = np.eye(num_classes)[test_labels].reshape(-1, num_classes)

In [11]:
# Create train and validation sets
val_split = 0.2
num_examples = train_data.shape[0]
val_size = int(val_split * num_examples)
train_size = num_examples - val_size

x_train = FLOAT_DTYPE(train_data[:train_size])
y_train = INT_DTYPE(train_labels_enc[:train_size])
x_val = FLOAT_DTYPE(train_data[train_size:])
y_val = INT_DTYPE(test_labels_enc[train_size:])

In [14]:
x_train = x_train.reshape(-1, 32, 32, 3)
x_val = x_val.reshape(-1, 32, 32, 3)

In [33]:
model = CNN()

model.add(Conv2D(3, 64))
model.add(BatchNorm())
model.add(Dropout(0.5))
model.add(Relu())

model.add(Conv2D(3, 128))
model.add(BatchNorm())
model.add(Dropout(0.5))
model.add(Relu())

model.add(Conv2D(3, 128))
model.add(BatchNorm())
model.add(Dropout(0.5))
model.add(Softmax())

In [34]:
model.compile(x_train.shape)

In [35]:
model.train(x_train, y_train, EPOCHS)

Epoch 1 / 100 in progress =========================================
Epoch 1 Forward Propogation Started


  0%|          | 0/12 [01:59<?, ?it/s]


MemoryError: Unable to allocate 17.2 GiB for an array with shape (40000, 30, 30, 64) and data type float64

In [25]:
from Utils.conv import asStride

_, w, h, c = (40000, 32, 32, 3)
size = (32, 32)
stride = (1, 1)

scale = np.sqrt(2. / (size[0] * size[1] * c))
weights = np.random.normal(loc=scale, scale=1., size=(
    size[0], size[1], c, 64))

bias = np.zeros(shape=(64, ), dtype=FLOAT_DTYPE)

kx, ky = size
sx, sy = stride
input = x_train.astype(FLOAT_DTYPE)

# If no pad, every image in the batch is cut
mat_pad = input[:, : (w - kx) // sx * sx + kx,
                : (h - ky) // sy * sy + ky, ...]

print(mat_pad.shape)

# Create the view of the array with shape (batch, out_w ,out_h, kx, ky, in_c)
view = asStride(mat_pad, size, stride)

print(view.shape)

# the choice of numpy.einsum is due to reshape of self.view being a copy
z = np.einsum('lmnijk, jkio -> lmno', view,
              weights, optimize=True) + bias


(40000, 32, 32, 3)
(40000, 1, 1, 3, 32, 32)


In [26]:
z.shape

(40000, 1, 1, 64)